In [1]:
import ROOT

OBJ: TStyle	ildStyle	ILD Style : 0 at: 0x8e64390


In [2]:
run_toy_diagnostics = False

In [3]:
# TODO: make it possible to fit multiple runs simultaneously
run = {
    "lumi": 5000,
    "e_pol": 0.,
    "p_pol": 0.,
}
n_bins = 65

In [4]:
parameters = {
    "g1z": 0.0,
    "ka": 0.0,
    "la": 0.0,
}
obs_names = [
    "O_g1z_pos_1em05",
    "O_ka_pos_1em05",
    "O_la_pos_1em05",
]
input_path = "data/histograms/full/raw_histograms.root"
signal_cat = "4f_sw_sl_signal"
signal_processes = [
    "4f_sw_sl_eLpL_signal",
    "4f_sw_sl_eLpR_signal",
    "4f_sw_sl_eRpL_signal",
    "4f_sw_sl_eRpR_signal",
    ]
background_processes = []

In [5]:
signal_histograms = {}
signal_meta = {}
oo_matrix = {}
template_parameters = {}
with ROOT.TFile(input_path) as input_file:
    # take care of signals first
    signal_dir = input_file[signal_cat]
    for process_name in signal_processes:
        signal_histos = {}
        p_dir = signal_dir[process_name]
        obs = "obs_ND"
        # very consistent ownership model of root requires us to do this
        histo = p_dir[obs]
        # not needed on nD???
        # histo.SetDirectory(ROOT.nullptr)
        signal_histo = histo
        meta_dir = p_dir["meta"]
        obs_meta = {}
        for key in meta_dir.GetListOfKeys():
            key_name = key.GetName()
            obs_meta[key_name] = meta_dir[key_name]
        signal_histograms[process_name] = signal_histo
        signal_meta[process_name] = obs_meta
        # get OO matrix
        mat_dir = input_file["oo_matrix"]
        oo_matrix[process_name] = mat_dir[process_name]
    # get template parameters
    # unfortunately here we have the loops the other way around
    template_dir = input_file["template_parametrisations"]
    for obs in obs_names:
        obs_dir = template_dir[obs]
        p_pars = {}
        for process_name in signal_processes:
            p_dir = obs_dir[process_name]
            pars = {}
            for par in parameters.keys():
            # very consistent ownership model of root requires us to do
                par_hist = p_dir[par]
                par_hist.SetDirectory(ROOT.nullptr)
                pars[par] = par_hist
            p_pars[process_name] = pars
        template_parameters[obs] = p_pars


    # TODO: implement background handling

print(signal_histograms)
print(signal_meta)
print(oo_matrix)
print(template_parameters)

{'4f_sw_sl_eLpL_signal': <cppyy.gbl.THnT<double> object at 0xabfc520>, '4f_sw_sl_eLpR_signal': <cppyy.gbl.THnT<double> object at 0xab83d40>, '4f_sw_sl_eRpL_signal': <cppyy.gbl.THnT<double> object at 0xace66e0>, '4f_sw_sl_eRpR_signal': <cppyy.gbl.THnT<double> object at 0xabbdc20>}
{'4f_sw_sl_eLpL_signal': {'lumi': <cppyy.gbl.TParameter<float> object at 0xad48160>, 'e_pol': <cppyy.gbl.TParameter<float> object at 0xad29160>, 'p_pol': <cppyy.gbl.TParameter<float> object at 0xad0d310>}, '4f_sw_sl_eLpR_signal': {'lumi': <cppyy.gbl.TParameter<float> object at 0xad41ec0>, 'e_pol': <cppyy.gbl.TParameter<float> object at 0xad63db0>, 'p_pol': <cppyy.gbl.TParameter<float> object at 0xad9da00>}, '4f_sw_sl_eRpL_signal': {'lumi': <cppyy.gbl.TParameter<float> object at 0xad09270>, 'e_pol': <cppyy.gbl.TParameter<float> object at 0xad7e290>, 'p_pol': <cppyy.gbl.TParameter<float> object at 0xad7e310>}, '4f_sw_sl_eRpR_signal': {'lumi': <cppyy.gbl.TParameter<float> object at 0xacebad0>, 'e_pol': <cppyy.gbl

In [6]:
from fit_utils import make_observed_histogram, hist_to_np, make_asimov_np, make_asimov_fast, make_1D_projections, make_observables, make_new_asimov_observables_np
h_obs = make_observed_histogram(signal_histograms, signal_meta, run)
h_obs_np = hist_to_np(h_obs)
h_asimov_np = make_asimov_np(h_obs_np)
h_asimov = make_asimov_fast(h_obs, 1234)

{ @0x7ffe8c8d9520, @0x7ffe8c8d9520, @0x7ffe8c8d9520 } 3


In [7]:
import numpy as np
h_example = make_1D_projections(h_obs)[0]
bin_centers = np.asarray([h_example.GetBinCenter(i+1) for i in range(h_example.GetNbinsX())])

In [8]:
signal_1d_histograms = [make_1D_projections(h) for h in signal_histograms.values()]
# ugly hack to "transpose" that list[list]
signal_1d_histograms = list(map(list, zip(*signal_1d_histograms)))
print(signal_1d_histograms)
signal_lumi = [par["lumi"].GetVal() for par in signal_meta.values()]
print(signal_lumi)
oo_matrices = list(oo_matrix.values())
print(oo_matrices)
template_param = [[[ph for ph in cpl_h.values()] for cpl_h in hel_h.values()] for hel_h in template_parameters.values()]
print(template_param)

[[<cppyy.gbl.TH1D object at 0xd4977e0>, <cppyy.gbl.TH1D object at 0xd2fcc30>, <cppyy.gbl.TH1D object at 0xd62ee10>, <cppyy.gbl.TH1D object at 0xd88d920>], [<cppyy.gbl.TH1D object at 0xd42de60>, <cppyy.gbl.TH1D object at 0xd30c0f0>, <cppyy.gbl.TH1D object at 0xd433c10>, <cppyy.gbl.TH1D object at 0xd4b7a10>], [<cppyy.gbl.TH1D object at 0xd353220>, <cppyy.gbl.TH1D object at 0xd41da30>, <cppyy.gbl.TH1D object at 0xd4b1a90>, <cppyy.gbl.TH1D object at 0xd415270>]]
[598.3264770507812, 100.83772277832031, 4009.4052734375, 696.610107421875]
[<cppyy.gbl.ROOT.Math.SVector<double,6> object at 0xabf66e0>, <cppyy.gbl.ROOT.Math.SVector<double,6> object at 0xacd06f0>, <cppyy.gbl.ROOT.Math.SVector<double,6> object at 0xad7e250>, <cppyy.gbl.ROOT.Math.SVector<double,6> object at 0xacebb50>]
[[[<cppyy.gbl.TH1D object at 0x8014750>, <cppyy.gbl.TH1D object at 0xae198e0>, <cppyy.gbl.TH1D object at 0x9db8fe0>], [<cppyy.gbl.TH1D object at 0x56f4ba0>, <cppyy.gbl.TH1D object at 0x568ace0>, <cppyy.gbl.TH1D object

In [9]:
ROOT.gInterpreter.Declare("#include \"fit.h\"")

True

In [10]:
fun2 = ROOT.fit_fun2[f"3, {n_bins}, 3"](signal_1d_histograms, signal_lumi, oo_matrices, template_param)

start constructor
finished constructor


In [11]:
if run_toy_diagnostics:
    n_obs = 3
    n_toys = 250000
    # h_chi2 = ROOT.TH1D("", ";#chi^{2}", 100, -10., 10.)
    h_chi2 = ROOT.TH1D("", ";#chi^{2}", 50, 0., 10.)
    h_prob = ROOT.TH1D("", ";probability", 50, 0., 1.)
    h_diff0 = ROOT.TH1D("", ";obs_asimov - obs [0]", 100, -10000., 10000.)
    h_diff1 = ROOT.TH1D("", ";obs_asimov - obs [1]", 100, -10000., 10000.)
    h_diff2 = ROOT.TH1D("", ";obs_asimov - obs [2]", 100, -10000., 10000.)
    obs_initial = make_observables(make_1D_projections(h_obs))
    C = np.zeros((n_obs, n_obs))
    v = np.zeros(n_obs)
    for seed in range(n_toys):
        # obs = make_new_asimov_observables(h_obs, seed)
        obs = make_new_asimov_observables_np(h_obs_np, bin_centers)
        pars = list(run.values()) + [0., 0., 0.] + [1.]
        chi2 = fun2(obs, pars)
        prob = ROOT.Math.chisquared_cdf(chi2, 3)
        h_chi2.Fill(chi2)
        h_prob.Fill(prob)
        diff = [0.] * n_obs
        for i in range(n_obs):
            diff[i] = obs[i] - obs_initial[i]
            v[i] += diff[i]
        for i in range(n_obs):
            for j in range(n_obs):
                C[i,j] += diff[i] * diff[j]
        h_diff0.Fill(diff[0])
        h_diff1.Fill(diff[1])
        h_diff2.Fill(diff[2])
    C /= n_toys
    v /= n_toys

In [ ]:
if run_toy_diagnostics:
    c_chi2 = ROOT.TCanvas()
    f = ROOT.TF1("chi2", "[0] * ROOT::Math::chisquared_pdf(x, [1])", 0., 10.)
    f.SetParameters(10000/5., 3.)
    # f.FixParameter(1, 3.)
    h_chi2.Fit(f)
    h_chi2.Draw()
    # f.Draw("same")
    c_chi2.Draw()
    # c_chi2.SaveAs("plots/fit/diagnostics/chi2.pdf")

    c_prob = ROOT.TCanvas()
    h_prob.SetMinimum(0)
    h_prob.Draw()
    c_prob.Draw()
    # c_prob.SaveAs("plots/fit/diagnostics/prob.pdf")

    c_diff0 = ROOT.TCanvas()
    h_diff0.Fit("gaus")
    h_diff0.Draw()
    c_diff0.Draw()
    # c_diff0.SaveAs("plots/fit/diagnostics/diff0.pdf")

    c_diff1 = ROOT.TCanvas()
    h_diff1.Fit("gaus")
    h_diff1.Draw()
    c_diff1.Draw()
    # c_diff1.SaveAs("plots/fit/diagnostics/diff1.pdf")

    c_diff2 = ROOT.TCanvas()
    h_diff2.Fit("gaus")
    h_diff2.Draw()
    c_diff2.Draw()
    # c_diff2.SaveAs("plots/fit/diagnostics/diff2.pdf")

In [13]:
# build all the RooFit stuff
# take one asimov obs and make it a RooDataset